In [ ]:
#Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
#SPDX-License-Identifier: MIT-0

# [Optional] Clean up
Execute the below cells if you want to delete your OpenSearch collection and associated encryption, networking and data access policies


In the following code cell, previously saved variables (`collection_id`, `encryption_policy_name`, and `network_policy_name`) are being loaded from the notebook's storage using the `%store -r` magic command in Jupyter Notebook. These variables were likely defined and stored in earlier code cells or notebooks, and are now being retrieved for use in subsequent computations or operations within this notebook.

In [ ]:
#loading saved variables from previous notebooks
%store -r collection_id
%store -r encryption_policy_name
%store -r network_policy_name
%store -r data_access_policy_name
%store -r data_access_policy_name_nb
%store -r aossAccessPolicyArn
%store -r identity_arn

In the following code cell, the code is setting up placeholders for certain variables: collection_id, encryption_policy_name, and network_policy_name. These variables are initially left empty as commented lines, allowing the user to manually set their values if needed. 

In [ ]:
#You can also manually set them if needed

#collection_id = ""
#encryption_policy_name = ""
#network_policy_name = ""

In the following code cell, an instance of the AWS boto3 client for OpenSearch Serverless is created. This client allows you to interact with the OpenSearch Serverless service, which is a fully managed search and analytics service provided by AWS. The code then prints a message indicating that it will delete the specified collection and associated policies, such as encryption policy, network policy, and data access policy, based on the provided identifiers and names.

In [ ]:
import boto3

# Create OpenSearch Serverless client
client = boto3.client('opensearchserverless')

print(f"You are about to delete the following collection: {collection_id}\nand the below policies:\n{encryption_policy_name},\n{network_policy_name},\n{data_access_policy_name}, \n{ data_access_policy_name_nb},\n{aossAccessPolicyArn}")

In the following code cell, we will delete opensearch collection

In [ ]:
import random
try:
    # Delete the collection
    response = client.delete_collection(
        clientToken='unique-token' + str(random.randint(1000, 9999)),
        id=collection_id
    )
    print(f"Collection deleted: {response['deleteCollectionDetail']['id']}")
except Exception as e:
    print(f"Error deleting collection: {e}")

In the following code cell, we will delete network policy for the Opensearch collection.

In [ ]:
# Delete the network policy
try:
    response = client.delete_security_policy(
        name=network_policy_name,
        type='network'
    )
    print("Network policy deleted")
except client.exceptions.ResourceNotFoundException:
    print("Network policy not found")
except Exception as e:
    print(f"Error deleting network policy: {e}")

In the following code cell, we will delete encryption policy for the Opensearch collection

In [ ]:
# Delete the encryption policy
try:
    response = client.delete_security_policy(
        name=encryption_policy_name,
        type='encryption'
    )
    print("Encryption policy deleted")
except client.exceptions.ResourceNotFoundException:
    print("Encryption policy not found")
except Exception as e:
    print(f"Error deleting encryption policy: {e}")

In the following code cell, we will delete data access policies for the Opensearch collection

In [ ]:
# Delete the data access policy
try:
    response = client.delete_access_policy(
        name=data_access_policy_name,
        type='data'
    )
    print("Data access policy deleted")
except client.exceptions.ResourceNotFoundException:
    print("Data access policy not found")
except Exception as e:
    print(f"Error deleting data access policy: {e}")    


In [ ]:
# Delete the data access policy used by notebook
try:
    response = client.delete_access_policy(
        name=data_access_policy_name_nb,
        type='data'
    )
    print("Data access policy deleted")
except client.exceptions.ResourceNotFoundException:
    print("Data access policy not found")
except Exception as e:
    print(f"Error deleting data access policy: {e}") 

In the following code cell, we will delete the AOSSAccessPolicy using IAM Client. TO delete the policy, first we will detach it from logged in user/role

In [ ]:
# Delete AOSSAccessPolicy

# Create an IAM client
iam = boto3.client('iam')

#First we will detach the aossaccesspolicy from the logged in user/role
try:
    # Try to get the user information
    user = iam.get_user(UserName=identity_arn.split('/')[-1])
    print(f"The identity ARN '{identity_arn}' is for a user.")

    # Detach the policy to the user
    iam.detach_user_policy(
        UserName=user['User']['UserName'],
        PolicyArn=aossAccessPolicyArn
    )

except iam.exceptions.NoSuchEntityException:
    # If the identity ARN is not for a user, it must be for a role
    print(f"The identity ARN '{identity_arn}' is for a role.")

    # Detach the policy from the role
    iam.detach_role_policy(
        RoleName=identity_arn.split('/')[-1],
        PolicyArn=aossAccessPolicyArn
    )

# Delete the aossaccesspolicy 
try:
    response = iam.delete_policy(
       PolicyArn=aossAccessPolicyArn
    )
    print("IAM policy deleted")
except client.exceptions.ResourceNotFoundException:
    print("IAM policy not found")
except Exception as e:
    print(f"Error deleting IAM policy: {e}")    